In [1]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 1

In [2]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())
adj1 = np.zeros_like(adj1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
beta = np.array([-1,2,0.1,2,0.1,-2,0.1,2,0.1,0.1])

Y_chain, A_chain, L_chain = sample_network_chain(adj1, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2, dir3, dir4 = [], [], [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(np.mean(Y[A==1]) - Y[A==0]))
    dir2.append(np.mean(A))
    dir3.append(np.mean(A[L[:,0]==1]) - np.mean(A[L[:,0]==0]))
    dir4.append(np.mean(Y[L[:,0]==1]) - np.mean(Y[L[:,0]==0]))

np.mean(dir), np.mean(dir2), np.mean(Y_chain), np.mean(dir3), np.mean(dir4)

1 9


100%|██████████| 320/320 [00:00<00:00, 405.41it/s]


(0.6279311431977952,
 0.35402500000000003,
 0.4556375,
 0.33663945218091595,
 0.380307566673866)

In [4]:
from drnet import expit
from sklearn.linear_model import LogisticRegression

def get_prob(L, Y, A):
    x_a = [L[:, 0], np.zeros(L.shape[0]), 
           L[:, 1], np.zeros(L.shape[0]), 
           L[:, 2], np.zeros(L.shape[0]), 
           np.zeros(L.shape[0])]
    
    # train lostic regression
    model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
    model.fit(np.array(x_a).T, A)
    
    gamma = np.zeros(8)
    gamma[0] = model.intercept_[0]
    gamma[1:] = model.coef_[0]

    #gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1]) 

    linpred = (
            gamma[0]
            + gamma[1] * L[:, 0] + gamma[2] * 0
            + gamma[3] * L[:, 1] + gamma[4] * 0
            + gamma[5] * L[:, 2] + gamma[6] * 0
            + gamma[7] * 0
        )
    
    probs = expit(linpred)

    w = (A==1) / probs
    w_norm = w/np.sum(w)*len(w) if np.sum(w) > 0 else 0
    w_norm = (A==1) # / probs
    gamma_1 = (Y * w_norm).mean()
    gamma_1_var = (Y * w_norm).var()

    w = (1-A) / (1-probs)
    w_norm = w/np.sum(w)*len(w) if np.sum(w) > 0 else 0
    w_norm = (1-A) # / (1-probs)
    gamma_0 = (Y * w_norm).mean()
    gamma_0_var = (Y * w_norm).var()

    return gamma_1, gamma_0, gamma_1_var, gamma_0_var

gamma_1_est, gamma_0_est = [], []
gamma_1_var, gamma_0_var = [], []
for i in range(Y_chain.shape[0]):
    gamma_1, gamma_0, gamma_1_var_, gamma0_var_ = get_prob(L_chain[i], Y_chain[i], A_chain[i])
    gamma_1_est.append(gamma_1)
    gamma_0_est.append(gamma_0)
    gamma_1_var.append(gamma_1_var_)
    gamma_0_var.append(gamma0_var_)



print(np.mean(gamma_1_est), np.mean(gamma_0_est))
print(np.var(gamma_1_est)*sample_size, np.var(gamma_0_est)*sample_size, np.mean(gamma_1_var), np.mean(gamma_0_var))
#get_prob(L_chain[2], Y_chain[2], A_chain[2])

0.304925 0.1507125
0.24367050000000007 0.108981375 0.21164115625000002 0.127862015625
